In [40]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import torchvision
from torchvision.transforms.v2 import PILToTensor
from pathlib import Path
import matplotlib.pyplot as plt
import os

In [44]:
data_path = Path("data")
image_path = data_path / "CIFAR10"
image_transform = torchvision.transforms.Compose([PILToTensor()])
train_data = torchvision.datasets.CIFAR10(root=image_path,
                                         train=True,
                                         download=True,
                                         transform=image_transform)
test_data = torchvision.datasets.CIFAR10(root=image_path,
                                         train=False,
                                         download=True,
                                        transform=image_transform)
print(train_data, test_data)

Files already downloaded and verified
Files already downloaded and verified
Dataset CIFAR10
    Number of datapoints: 50000
    Root location: data\CIFAR10
    Split: Train
    StandardTransform
Transform: Compose(
               PILToTensor()
           ) Dataset CIFAR10
    Number of datapoints: 10000
    Root location: data\CIFAR10
    Split: Test
    StandardTransform
Transform: Compose(
               PILToTensor()
           )


In [46]:
BATCH_SIZE = 32
NUM_WORKERS = os.cpu_count()

train_dataloader = DataLoader(dataset=train_data,
                              batch_size=BATCH_SIZE,
                              shuffle=True,
                              num_workers=NUM_WORKERS)
test_dataloader = DataLoader(dataset=test_data,
                             batch_size=BATCH_SIZE,
                             shuffle=False,
                             num_workers=NUM_WORKERS)
print(train_dataloader, test_dataloader)
print(f"Train dataloader lenght: {len(train_dataloader)}\nTest dataloader lenght: {len(test_dataloader)}")

<torch.utils.data.dataloader.DataLoader object at 0x0000029000DC3700> <torch.utils.data.dataloader.DataLoader object at 0x0000029000DC3820>
Train dataloader lenght: 1563
Test dataloader lenght: 313


In [48]:
image_batch, label_batch = next(iter(train_dataloader))
image_batch.shape

torch.Size([32, 3, 32, 32])